In [15]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from scipy.fftpack import ifftshift,fftshift,fft,ifft
from scipy.linalg import circulant,toeplitz, hankel, expm
from matplotlib import colors
import pickle
import sys
from numba import jit


In [6]:
class Sampling_Random_State:
    #### --------- Definition of variables ------------------------
    N_size=500001
    Gamma=0.5
    Lambda=0.5
    num_data = 2000
    #### ------------------------------------------------------------


    @classmethod
    def Alpha(cls,theta:np.float64)-> np.float64:
        return cls.Lambda+np.cos(theta)
    @classmethod
    def Beta(cls,theta:np.float64)-> np.float64:
        return cls.Gamma*np.sin(theta)
    @classmethod
    def Omega(cls,theta:np.float64)-> np.float64:
        return np.sqrt(cls.Alpha(theta)**2 + cls.Beta(theta)**2 )
    @classmethod
    def Phi(cls,theta:np.float64)-> np.float64:
        return np.arctan2(cls.Beta(theta),cls.Alpha(theta))



    @classmethod
    def Fermi_dirac(cls,n:np.int64,mu:np.float64 =0) -> np.float64:
        # beta is the inverse thermic energy associated in the system (beta)
        # mu corresponds to the chemical potential
        # n is the position of the particle
        # f=np.exp(T*(Omega(Gamma,Lambda,2.0*(np.pi/N)*n)-mu)) +1
        # N corresponds to the size of the system
        beta = np.min(cls.Omega(np.linspace(-np.pi,np.pi,int(1000))))
        f=np.exp(beta*(cls.Omega(((2.*np.pi)/np.float64(cls.N_size)) * n)-mu)) +1
        return 1/f

    @classmethod
    def Sample_State(cls,Ground:bool =False,mu:np.float64 = 0.0)-> np.ndarray:

        if Ground:
            x=np.arange(0,(cls.N_size-1)/2+ 1)
            m_cos=[-0.5 for i in x]
            m_sin=[-0.5 for i in x]
            x=np.arange(-(cls.N_size-1)/2,(cls.N_size-1)/2+1)
            M_minous=[((m_cos[np.abs(int(i))]-m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
            M_plus = [((m_cos[np.abs(int(i))]+m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
            Mminousband=np.array(M_minous)
            Mplusband=np.array(M_plus)
        else:
            x=np.arange(0,(cls.N_size-1)/2+ 1)
            m_cos=[-0.5 if np.random.random()>cls.Fermi_dirac(mu=mu,n=i) else 0.5 for i in x]
            m_sin=[-0.5 if np.random.random()>cls.Fermi_dirac(mu=mu,n=i) else 0.5 for i in x]
            x=np.arange(-(cls.N_size-1)/2,(cls.N_size-1)/2+1)
            M_minous=[((m_cos[np.abs(int(i))]-m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
            M_plus = [((m_cos[np.abs(int(i))]+m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
            Mminousband=np.array(M_minous)
            Mplusband=np.array(M_plus)
        return Mminousband,Mplusband

In [ ]:
@jit(nopython=True)

In [5]:
a= Sampling_Random_State()

In [89]:
%%timeit 
a.Sample_State()

6.08 ms ± 635 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [77]:
N_size=5001
#N_size=51
Lambda=0.5
Gamma=0.5
@jit(nopython=True)
def Alpha(theta:np.float64)-> np.float64:
    return Lambda+np.cos(theta)
@jit(nopython=True)
def Beta(theta:np.float64)-> np.float64:
    return Gamma*np.sin(theta)
@jit(nopython=True)
def Omega(theta:np.float64)-> np.float64:
    return np.sqrt(Alpha(theta)**2 + Beta(theta)**2 )
@jit(nopython=True)
def Phi(theta:np.float64)-> np.float64:
    return np.arctan2(Beta(theta),Alpha(theta))
@jit(nopython=True)
def Fermi_dirac(n:np.int64,mu:np.float64 =0) -> np.float64:
        # beta is the inverse thermic energy associated in the system (beta)
        # mu corresponds to the chemical potential
        # n is the position of the particle
        # f=np.exp(T*(Omega(Gamma,Lambda,2.0*(np.pi/N)*n)-mu)) +1
        # N corresponds to the size of the system
        beta = np.min(Omega(np.linspace(-np.pi,np.pi,int(1000))))
        f=np.exp(beta*(Omega(((2.*np.pi)/np.float64(N_size)) * n)-mu)) +1
        return 1/f

In [64]:
np.int64(32)

32

In [83]:
N_size=500001
@jit(nopython=True,parallel=True,fastmath = True)
def test():
    m_cos = np.zeros((N_size-1)//2+1,dtype=np.float64)
    m_sin = np.zeros((N_size-1)//2+1,dtype=np.float64)
    for index,i in enumerate(range(int((N_size-1)//2)+1)):
        randoms = np.random.random(2)
        if randoms[0] > Fermi_dirac(mu=0,n=np.int64(i)):
            m_cos[index]=-0.5 
        else:
            m_cos[index]=0.5
        if randoms[1] > Fermi_dirac(mu=0,n=np.int64(i)):
            m_sin[index]=-0.5
        else:
            m_sin[index]=0.5
    return m_sin, m_cos
    #Mplus= np.zeros(N_size,dtype=np.complex)
    #Mminous= np.zeros(N_size,dtype=np.complex)
    #for index,i in enumerate(np.arange(-(N_size-1)/2,(N_size-1)/2+1)):
    #    Mplus[index] = 0.5*(m_cos[np.abs(int(i))] + m_sin[np.abs(int(i))])*np.exp(1.j*np.sign((2.0*np.pi/N_size) * i)*Phi(np.abs((2.0*np.pi/N_size) * i)))
    #    Mminous[index] = 0.5*(m_cos[np.abs(int(i))] - m_sin[np.abs(int(i))])*np.exp(1.j*np.sign((2.0*np.pi/N_size) * i)*Phi(np.abs((2.0*np.pi/N_size) * i)))
    #return Mplus,Mminous
    
    
        

In [84]:
import time

In [86]:
start = time.time()
test()
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 19.312328100204468


In [87]:
# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
test()
end = time.time()

In [13]:
x=np.arange(0,(500001-1)/2+ 1)
cosa=map(a.Fermi_dirac,x)

In [28]:
algo=map(a.Fermi_dirac,x)

In [29]:
np.array(list(algo))

array([0.44914523, 0.44914523, 0.44914523, ..., 0.44914523, 0.44914523,
       0.44914523])

In [12]:
len(x)

21

In [10]:
for i in x:
    print(i)

-10.0
-9.0
-8.0
-7.0
-6.0
-5.0
-4.0
-3.0
-2.0
-1.0
0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0


In [40]:
np.roll(x,(21-1)//2 + 1)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
       -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.])

In [35]:
ifftshift(x)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
       -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.])

In [41]:
np.roll(x,-(21-1)//2 )

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
       -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.])

In [42]:
a=True

In [43]:
type(True)

bool

In [14]:
15//2

7

In [1]:
from scipy.fftpack import ifftshift,fftshift,fft,ifft
import numpy as np

In [2]:
N_size=5000001
x=np.arange(-(N_size-1)/2,(N_size-1)/2+1)

In [3]:
%%timeit
np.fft.fft(np.roll(x,-(N_size-1)//2))

1.72 s ± 8.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
fft(ifftshift(x))

1min 6s ± 2.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
np.fft.fft(ifftshift(x))

1.82 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
fft(np.roll(x,-(N_size-1)//2))

# Timing the different options

In [9]:
import numpy as np
from scipy.fftpack import ifftshift,fftshift,fft,ifft
class Sampling_Random_State:
    #### --------- Definition of variables ------------------------
    N_size=500001
    Gamma=0.5
    Lambda=0.5
    num_data = 2000
    #### ------------------------------------------------------------


    @classmethod
    def Alpha(cls,theta:np.float64)-> np.float64:
        return cls.Lambda+np.cos(theta)
    @classmethod
    def Beta(cls,theta:np.float64)-> np.float64:
        return cls.Gamma*np.sin(theta)
    @classmethod
    def Omega(cls,theta:np.float64)-> np.float64:
        return np.sqrt(cls.Alpha(theta)**2 + cls.Beta(theta)**2 )
    @classmethod
    def Phi(cls,theta:np.float64)-> np.float64:
        return np.arctan2(cls.Beta(theta),cls.Alpha(theta))



    @classmethod
    def Fermi_dirac(cls,n:np.int64,mu:np.float64 =0) -> np.float64:
        # beta is the inverse thermic energy associated in the system (beta)
        # mu corresponds to the chemical potential
        # n is the position of the particle
        # f=np.exp(T*(Omega(Gamma,Lambda,2.0*(np.pi/N)*n)-mu)) +1
        # N corresponds to the size of the system
        beta = np.min(cls.Omega(np.linspace(-np.pi,np.pi,int(1000))))
        f=np.exp(beta*(cls.Omega(((2.*np.pi)/np.float64(cls.N_size)) * n)-mu)) +1
        return 1/f

    @classmethod
    def Sample_State(cls,Ground:bool =False,mu:np.float64 = 0.0)-> np.ndarray:
        x=np.arange(0,(cls.N_size-1)/2+ 1)
        if Ground:
            m_cos=[-0.5 for i in x]
            m_sin=[-0.5 for i in x]
        else:
            m_cos=[-0.5 if np.random.random()>cls.Fermi_dirac(mu=mu,n=i) else 0.5 for i in x]
            m_sin=[-0.5 if np.random.random()>cls.Fermi_dirac(mu=mu,n=i) else 0.5 for i in x]
        x=np.arange(-(cls.N_size-1)/2,(cls.N_size-1)/2+1)
        M_minous=[((m_cos[np.abs(int(i))]-m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
        M_plus = [((m_cos[np.abs(int(i))]+m_sin[np.abs(int(i))])*0.5*np.exp(1.j*np.sign((2.0*np.pi/cls.N_size) * i)*cls.Phi(np.abs((2.0*np.pi/cls.N_size) * i)))) for i in x]
        Mminousband=np.array(M_minous)
        Mplusband=np.array(M_plus)
        return Mminousband,Mplusband

In [ ]:
a=Sampling_Random_State()
M_m, M_p = a.Sample_State()

In [12]:
%%timeit
M_m, M_p = a.Sample_State()

47.9 s ± 95.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
fft(ifftshift(M_p))

In [ ]:
%%timeit
np.fft.fft(np.roll(M_p,-(a.N_size-1)//2))

In [2]:
N_size=51
x=np.arange(-(N_size-1)/2,(N_size-1)/2+1)
np.fft.ifftshift(x)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25., -25., -24., -23., -22., -21., -20., -19.,
       -18., -17., -16., -15., -14., -13., -12., -11., -10.,  -9.,  -8.,
        -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.])

In [6]:
from pyfftw.interfaces import scipy_fftpack

In [7]:
scipy_fftpack.fft(M_p)

NameError: name 'M_p' is not defined